# Machine Learning - Logistic Regression

## Dataset

In [1]:
import pandas as pd
import numpy as np

train_data = pd.read_csv('../data/raw/dataset_train.csv').dropna()

# Remove the columns that are not features and standardize the data
def standardize(x):
    means = np.mean(x, axis=0)
    stds = np.std(x, axis=0)

    stds[stds == 0] = 1
    return (x - means) / stds
train_data_filtered = train_data.drop(columns = ['Hogwarts House', 'First Name', 'Last Name', 'Birthday', 'Best Hand', 'Index', 'Astronomy', 'Arithmancy', 'Care of Magical Creatures'])
x_train_standardized = standardize(train_data_filtered)

y_true = {
    'gryffindor': (train_data['Hogwarts House'] == 'Gryffindor').astype(int),
    'slytherin': (train_data['Hogwarts House'] == 'Slytherin').astype(int),
    'ravenclaw': (train_data['Hogwarts House'] == 'Ravenclaw').astype(int),
    'hufflepuff': (train_data['Hogwarts House'] == 'Hufflepuff').astype(int)
}
train_data

,Index,Hogwarts House,First Name,Last Name,Birthday,Best Hand,Arithmancy,Astronomy,Herbology,Defense Against the Dark Arts,Divination,Muggle Studies,Ancient Runes,History of Magic,Transfiguration,Potions,Care of Magical Creatures,Charms,Flying
0,0,Ravenclaw,Tamara,Hsu,2000-03-30,Left,58384.0,-487.886086,5.727180,4.878861,4.722,272.035831,532.484226,5.231058,1039.788281,3.790369,0.715939,-232.79405,-26.89
1,1,Slytherin,Erich,Paredes,1999-10-14,Right,67239.0,-552.060507,-5.987446,5.520605,-5.612,-487.340557,367.760303,4.107170,1058.944592,7.248742,0.091674,-252.18425,-113.45
2,2,Ravenclaw,Stephany,Braun,1999-11-03,Left,23702.0,-366.076117,7.725017,3.660761,6.140,664.893521,602.585284,3.555579,1088.088348,8.728531,-0.515327,-227.34265,30.42
3,3,Gryffindor,Vesta,Mcmichael,2000-08-19,Left,32667.0,697.742809,-6.497214,-6.977428,4.026,-537.001128,523.982133,-4.809637,920.391449,0.821911,-0.014040,-256.84675,200.64
5,5,Slytherin,Corrine,Hammond,1999-04-04,Right,21209.0,-613.687160,-4.289197,6.136872,-6.592,-440.997704,396.201804,5.380286,1052.845164,11.751212,1.049894,-247.94549,-34.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,1595,Gryffindor,Jung,Blank,2001-09-14,Right,49009.0,354.280086,-4.541837,-3.542801,5.702,-497.235066,618.220213,-5.231721,964.219853,3.389086,-0.649983,-250.39401,185.83
1596,1596,Slytherin,Shelli,Lock,1998-03-12,Left,63296.0,367.531174,6.061064,-3.675312,1.757,-643.271092,445.827565,2.238112,1056.147366,5.825263,-0.333962,-246.42719,44.80
1597,1597,Gryffindor,Benjamin,Christensen,1999-10-24,Right,63905.0,544.018925,-3.203269,-5.440189,6.065,-385.150457,635.211486,-5.984257,953.866685,1.709808,0.071569,-251.63679,198.47
1598,1598,Hufflepuff,Charlotte,Dillon,2001-09-21,Left,82713.0,453.676219,3.442831,-4.536762,6.738,-831.741123,383.444937,3.813111,1087.949205,3.904100,-0.531875,-246.19072,-76.81


## Logistic Regression
### Log loss
The log loss function is used to evaluate the performance of a classification model. It is defined as follows
With y_true representing the true labels and y_pred representing the predicted labels.
The log loss is also known as the cross-entropy loss.

In [7]:
def log_loss(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7) # Clip the values to avoid log(0)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Example log loss
y_true_sample = np.array([0, 1, 1, 0, 1])
y_pred_sample = np.array([0.1, 0.9, 0.8, 0.2, 0.3])
print(log_loss(y_true_sample, y_pred_sample))

0.3721961876540016


### Hypothesis
The hypothesis function for logistic regression is defined as follows
With theta represents the weights of the features and x represents the features of the sample.

In [8]:
def hypothesis(x, theta):
    return 1 / (1 + np.exp(-x @ theta))

# Example hypothesis one feature
x = np.array([2])
theta = np.array([0.5])
print(hypothesis(x, theta))

# Example hypothesis multiple features
x = np.array([1, 3, 4]) #  1 = bias , 3 = feature_1_value, 4 = feature_2_value
theta = np.array([0.5, 0.1, 0.2]) # 0.5 = bias, 0.1 = feature_1_weight, 0.2 = feature_2_weight
print(hypothesis(x, theta))

# Example hypothesis multiple features and multiple samples
x = np.array([[1, 3, 4], [1, 2, 5], [1, 1, 6]]) #  1,3,4 = bias , 1,2,5 = feature_1_value, 1,1,6 = feature_2_value
theta = np.array([0.5, 0.1, 0.2]) # 0.5 = bias, 0.1 = feature_1_weight, 0.2 = feature_2_weight
print(hypothesis(x, theta))

0.7310585786300049
0.8320183851339245
[0.83201839 0.84553473 0.85814894]


## Gradient
### Stochastic Gradient Descent

In [23]:
def stochastic_gradient_descent(x, y_true, theta, learning_rate, epochs):
    m = x.shape[0]
    print(f'Training started: {m} samples, {epochs} epochs, learning rate: {learning_rate}..., theta: {theta[:5]}...')
    theta_train_loop = theta.copy()
    for epoch in range(epochs):
        for i in range(m):
            random_index = np.random.randint(m)
            x_i = x[random_index:random_index + 1]
            y_i = y_true[random_index:random_index + 1]
            y_pred = hypothesis(x_i, theta_train_loop)
            gradient = x_i.T @ (y_pred - y_i)
            theta_train_loop -= learning_rate * gradient
    print(f'Training finished: {log_loss(y_true, hypothesis(x, theta_train_loop))} log loss, {theta_train_loop[:5]}... theta \n')
    return theta_train_loop

bias = np.ones((x_train_standardized.shape[0], 1)) # Create a column of ones for the bias, with x_train.shape[0] = number of rows, so we create a column of ones with the same number of rows as x_train plus one column, the bias
x_bias = np.hstack([bias, x_train_standardized]) # Add the bias column to the x_train
initial_theta = np.zeros(x_bias.shape[1])
theta_train = {
    'gryffindor': stochastic_gradient_descent(x_bias, y_true['gryffindor'], initial_theta , 0.1, 100),
    'slytherin': stochastic_gradient_descent(x_bias, y_true['slytherin'], initial_theta, 0.1, 100),
    'ravenclaw': stochastic_gradient_descent(x_bias, y_true['ravenclaw'], initial_theta, 0.1, 100),
    'hufflepuff': stochastic_gradient_descent(x_bias, y_true['hufflepuff'], initial_theta, 0.1, 100)
}

print("To conclude, the final theta is: ")
theta_train
# features = train_data_filtered.columns.insert(0, 'Bias')
# theta_dict = dict(zip(features, theta_train))
# theta_dict

Training started: 1251 samples, 100 epochs, learning rate: 0.1..., theta: [0. 0. 0. 0. 0.]...
Training finished: 0.043928097381231906 log loss, [-3.49393424 -1.42931541  0.08829902  2.65178519 -0.98121965]... theta 

Training started: 1251 samples, 100 epochs, learning rate: 0.1..., theta: [0. 0. 0. 0. 0.]...
Training finished: 0.03224474011334483 log loss, [-5.01218535 -0.12494031  1.7422377  -2.69615217 -1.34418688]... theta 

Training started: 1251 samples, 100 epochs, learning rate: 0.1..., theta: [0. 0. 0. 0. 0.]...
Training finished: 0.06884723621325846 log loss, [-2.54566817  0.69361317  1.26035852 -0.16782006  1.98025976]... theta 

Training started: 1251 samples, 100 epochs, learning rate: 0.1..., theta: [0. 0. 0. 0. 0.]...
Training finished: 0.05539857675761649 log loss, [-2.75768722  2.20759903 -2.47567216  0.49343251 -1.20589351]... theta 

To conclude, the final theta is: 


{'gryffindor': array([-3.49393424, -1.42931541,  0.08829902,  2.65178519, -0.98121965,
         0.29406253, -0.71393398, -1.52340126,  0.02341803,  0.87885243,
         1.69819368]),
 'slytherin': array([-5.01218535, -0.12494031,  1.7422377 , -2.69615217, -1.34418688,
        -0.37271187, -0.61308514,  1.26330958,  1.37367699, -0.45238373,
         0.02434027]),
 'ravenclaw': array([-2.54566817,  0.69361317,  1.26035852, -0.16782006,  1.98025976,
         1.53430769,  0.16579869,  0.01871794, -0.29523291,  1.10897304,
        -0.34593762]),
 'hufflepuff': array([-2.75768722,  2.20759903, -2.47567216,  0.49343251, -1.20589351,
        -0.8207735 ,  1.70485264,  0.16457834, -0.81004128, -0.8596664 ,
        -1.0648127 ])}

### Store theta train

In [24]:
import pickle

def store_theta_train(theta_train):
    # Store data (serialize)
    with open('../results/models/theta_train.pkl', 'wb') as handle:
        pickle.dump(theta_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
        print("Store theta train:", theta_train)

def load_theta_train():
    # Load data (deserialize)
    with open('../results/models/theta_train.pkl', 'rb') as handle:
        theta_train = pickle.load(handle)
        print("Theta_train loaded:", theta_train)
        return theta_train
        
store_theta_train(theta_train)
# theta_train = load_theta_train()


Store theta train: {'gryffindor': array([-3.49393424, -1.42931541,  0.08829902,  2.65178519, -0.98121965,
        0.29406253, -0.71393398, -1.52340126,  0.02341803,  0.87885243,
        1.69819368]), 'slytherin': array([-5.01218535, -0.12494031,  1.7422377 , -2.69615217, -1.34418688,
       -0.37271187, -0.61308514,  1.26330958,  1.37367699, -0.45238373,
        0.02434027]), 'ravenclaw': array([-2.54566817,  0.69361317,  1.26035852, -0.16782006,  1.98025976,
        1.53430769,  0.16579869,  0.01871794, -0.29523291,  1.10897304,
       -0.34593762]), 'hufflepuff': array([-2.75768722,  2.20759903, -2.47567216,  0.49343251, -1.20589351,
       -0.8207735 ,  1.70485264,  0.16457834, -0.81004128, -0.8596664 ,
       -1.0648127 ])}


### Predict

In [56]:
def predict(x, theta):
    bias = np.ones((x.shape[0], 1))
    x_bias = np.hstack([bias, x])
    probabilities = {
        'gryffindor': hypothesis(x_bias, theta['gryffindor']),
        'slytherin': hypothesis(x_bias, theta['slytherin']),
        'ravenclaw': hypothesis(x_bias, theta['ravenclaw']),
        'hufflepuff': hypothesis(x_bias, theta['hufflepuff'])
    }
    return pd.DataFrame(probabilities)

predict(x_train_standardized, theta_train)

,gryffindor,slytherin,ravenclaw,hufflepuff
0,0.005994,0.000634,0.974236,0.023482
1,0.000011,0.997004,0.004863,0.004103
2,0.005530,0.001438,0.997832,0.001417
3,0.998456,0.000031,0.000263,0.007349
4,0.000024,0.999604,0.011709,0.000551
...,...,...,...,...
1246,0.997998,0.000168,0.006202,0.000568
1247,0.002910,0.050052,0.003950,0.881303
1248,0.998389,0.000020,0.008622,0.001880
1249,0.005656,0.002752,0.000716,0.995412


## Compare with sklearn

In [67]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

sgd = SGDClassifier(loss='log_loss', learning_rate='constant', max_iter=100, tol=1e-7, random_state=42, eta0=0.1)
sgd.fit(x_train_standardized, y_true['gryffindor'])
print(sgd.coef_[0])
theta_train['gryffindor'] - np.hstack([sgd.intercept_, sgd.coef_[0]])
x_test = pd.read_csv('../data/raw/dataset_test.csv')
x_test_filtered = x_test.drop(columns = ['First Name', 'Last Name', 'Birthday', 'Best Hand', 'Index', 'Astronomy', 'Arithmancy', 'Care of Magical Creatures', 'Hogwarts House']).dropna()

# accuracy_score(sgd.predict(standardize(x_test_filtered)), predict(x_test_filtered, theta_train)['gryffindor'])
# predict(standardize(x_test_filtered), theta_train)['gryffindor']
binary_predictions = predict(standardize(x_test_filtered), theta_train)['gryffindor'].apply(lambda x: 1 if x > 0.5 else 0)
print(accuracy_score(sgd.predict(standardize(x_test_filtered)), binary_predictions))

[-1.16222735 -0.48711888  1.48263139 -0.31693973  0.86561988 -0.57019407
 -1.32198748  0.20903784 -0.00532947  1.09389221]
1.0


## Optimisation bonus : mini-batch gd 

In [12]:
def mini_batch_gradient_descent(x, y_true, theta, learning_rate, epochs, batch_size):
    m = x.shape[0]
    print(f'Training started: {m} samples, {epochs} epochs, learning rate: {learning_rate}, batch size: {batch_size}..., theta: {theta[:5]}...')
    theta_train_loop = theta.copy()
    for epoch in range(epochs):
        for i in range(0, m, batch_size):
            x_i = x[i:i + batch_size]
            y_i = y_true[i:i + batch_size]
            y_pred = hypothesis(x_i, theta_train_loop)
            gradient = x_i.T @ (y_pred - y_i)
            theta_train_loop -= learning_rate * gradient
    print(f'Training finished: {log_loss(y_true, hypothesis(x, theta_train_loop))} log loss, {theta_train_loop[:5]}... theta \n')
    return theta_train_loop
bias = np.ones((x_train_standardized.shape[0], 1)) # Create a column of ones for the bias, with x_train.shape[0] = number of rows, so we create a column of ones with the same number of rows as x_train plus one column, the bias
x_bias = np.hstack([bias, x_train_standardized]) # Add the bias column to the x_train
initial_theta = np.zeros(x_bias.shape[1])
mini_batch_gradient_descent(x_bias, y_true['gryffindor'], initial_theta, 0.1, 100, 32)

Training started: 1251 samples, 100 epochs, learning rate: 0.1, batch size: 32..., theta: [0. 0. 0. 0. 0.]...
Training finished: 0.04203245972395851 log loss, [-3.68655132 -1.60916743 -0.02186867  2.4397184  -0.49515849]... theta 


array([-3.68655132, -1.60916743, -0.02186867,  2.4397184 , -0.49515849,
        0.06369299, -0.41579199, -1.63988086,  0.23750852,  0.83526254,
        1.54687037])